In [ ]:
import pinocchio as pin

In [ ]:
import numpy as np

In [30]:
q = np.deg2rad([0.0, 15.0, 0.0, -75.0, 0.0, 90.0, 0.0])
nv = 7
v = np.zeros(nv)
vd = np.zeros(nv)

In [65]:
from pydrake.all import Parser, MultibodyPlant
plant = MultibodyPlant(time_step=0.0)
parser = Parser(plant)
urdf_uri = "package://drake/manipulation/models/franka_description/urdf/panda_arm.urdf"
parser.AddModelsFromUrl(urdf_uri)
plant.WeldFrames(
    plant.world_frame(),
    plant.GetFrameByName("panda_link0"),
)
plant.Finalize()
context = plant.CreateDefaultContext()
plant.SetPositions(context, q)
M_d = plant.CalcMassMatrix(context)
g_d = plant.CalcGravityGeneralizedForces(context)
print(M_d)

[[ 1.52172607e+00 -5.42016772e-02  1.15962793e+00  1.20886255e-02
   1.37462908e-02  4.52922634e-04 -4.59801871e-03]
 [-5.42016772e-02  2.56120153e+00 -6.87017423e-02 -1.13634711e+00
  -6.84840311e-02 -8.28314924e-02 -1.66345636e-03]
 [ 1.15962793e+00 -6.87017423e-02  9.11346473e-01  2.32927396e-02
   1.67794228e-02  2.01545845e-04 -3.96903652e-03]
 [ 1.20886255e-02 -1.13634711e+00  2.32927396e-02  7.17084783e-01
   4.50791421e-02  6.73871081e-02  8.40479598e-06]
 [ 1.37462908e-02 -6.84840311e-02  1.67794228e-02  4.50791421e-02
   2.12245918e-02 -9.11616931e-04 -3.45402924e-04]
 [ 4.52922634e-04 -8.28314924e-02  2.01545845e-04  6.73871081e-02
  -9.11616931e-04  1.82121924e-02 -4.72579279e-04]
 [-4.59801871e-03 -1.66345636e-03 -3.96903652e-03  8.40479598e-06
  -3.45402924e-04 -4.72579279e-04  8.45606406e-04]]


In [32]:
# should remove hand
urdf_file = "./data/pinocchio_robots/panda_description/urdf/panda.urdf"

# can't get filepath from Drake; also, can't have pinnochio load the model :shrug:
# # package_map = parser.package_map()
# # print(package_map.GetPackageNames())
# # package_map.GetPath(urdf_uri
# # urdf_file = "~/.cache/pypoetry/virtualenvs/tsid-control-i1cFkKOL-py3.10/lib/python3.10/site-packages/pydrake/share/drake/manipulation/models/
# franka_description/urdf/panda_arm.urdf"

model = pin.buildModelFromUrdf(urdf_file)

In [35]:
data = model.createData()
# nv = model.nv
# q = pin.randomConfiguration(model)

In [36]:
tau_c = np.zeros(nv)
pin.aba(model, data, q, v, tau_c)
pin.computeMinverse(model, data, q);
data.M

# Doesn't make sense - not sure what I'm doing wrong :(

array([[0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.]])

In [10]:
import mujoco

In [59]:
# er, uses inertias from the sys id paper which may not be spot-on.
model = mujoco.MjModel.from_xml_path("./data/mujoco_menagerie/franka_emika_panda/panda_nohand.xml")
data = mujoco.MjData(model)

In [60]:
# https://github.com/ir-lab/irl_control/blob/a232828a53635cb2dba5be313f2a57227c10e18e/irl_control/osc.py#L101

In [63]:
data.qpos[:] = q
data.qvel[:] = v
data.qacc[:] = vd
# mujoco.mj_i (model, data)
mujoco.mj_inverse(model, data)

In [64]:
M_m = np.zeros((nv, nv))
mujoco.mj_fullM(model, M_m, data.qM)
g_m = data.qfrc_bias

In [66]:
print(M_m - M_d)
print(g_m - g_d)

[[ 2.23086767e-01  1.67571917e-02  1.00079124e-01 -1.10099562e-02
   2.75907850e-02  1.89005566e-04 -5.21187495e-03]
 [ 1.67571917e-02  2.21508101e-01  2.59147753e-02 -8.11038801e-02
   3.21440136e-02 -1.79058444e-02  3.35764674e-03]
 [ 1.00079124e-01  2.59147753e-02  1.94177427e-01 -1.62381821e-02
   2.54318813e-02  3.60623547e-04 -5.20593748e-03]
 [-1.10099562e-02 -8.11038801e-02 -1.62381821e-02  1.70886628e-01
  -2.02035878e-02  2.50031127e-02 -8.14777661e-04]
 [ 2.75907850e-02  3.21440136e-02  2.54318813e-02 -2.02035878e-02
   1.07445388e-01  6.87964014e-04 -1.32707976e-03]
 [ 1.89005566e-04 -1.79058444e-02  3.60623547e-04  2.50031127e-02
   6.87964014e-04  1.13538760e-01 -7.57798237e-05]
 [-5.21187495e-03  3.35764674e-03 -5.20593748e-03 -8.14777661e-04
  -1.32707976e-03 -7.57798237e-05  1.04064046e-01]]
[ 0.00000000e+00 -6.50775379e+01  6.18470348e-01  3.60557504e+01
  2.11838900e+00  2.99475229e+00  5.49693162e-13]
